In [7]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20,256),
                    nn.ReLU(),
                    nn.Linear(256,10))

X = torch.rand(2,20)
print(X)
# 相当于net.__call__(X)
net(X)

tensor([[0.9203, 0.0238, 0.4650, 0.8131, 0.4727, 0.1956, 0.5542, 0.7496, 0.8125,
         0.1979, 0.1056, 0.0906, 0.8445, 0.2159, 0.1264, 0.6718, 0.8577, 0.7936,
         0.5812, 0.9574],
        [0.1153, 0.5230, 0.4647, 0.1595, 0.9942, 0.9747, 0.4044, 0.6720, 0.3382,
         0.7251, 0.4679, 0.4253, 0.2344, 0.6465, 0.3727, 0.4963, 0.3079, 0.7314,
         0.3474, 0.6225]])


tensor([[ 0.0609, -0.0083,  0.0305,  0.0665, -0.2251,  0.0904,  0.2813,  0.1470,
         -0.2698,  0.0417],
        [ 0.0427, -0.0208, -0.1815, -0.0108, -0.1847, -0.0030,  0.0980,  0.0546,
         -0.2780, -0.0364]], grad_fn=<AddmmBackward0>)

# 实现一个block

In [8]:
class MLP(nn.Module):
  # 用模型参数声明层，这里声明两个全连接层
  def __init__(self):
    super().__init__()
    self.hidden = nn.Linear(20,256) #隐藏层
    self.out = nn.Linear(256,10)    #输出层
  
  # 定义模型的前向传播
  def forward(self,X):
    # 使用nn.functional中的relu，这是函数版本
    return self.out(F.relu(self.hidden(X)))
    

In [9]:
net = MLP()
net(X)

tensor([[ 0.0914, -0.2605,  0.1241, -0.0729, -0.2081, -0.0296,  0.0875, -0.3039,
          0.0111,  0.2279],
        [-0.0261, -0.1974,  0.0953, -0.0284, -0.2870,  0.0208,  0.0910, -0.2241,
         -0.0163,  0.2424]], grad_fn=<AddmmBackward0>)

## 构建Sequential

In [12]:
class MySequential(nn.Module):
  def __init__(self, *args):
    super().__init__()
    for idx , module in enumerate(args):
      # 这里，module是Module子类的一个实例，保存在Module成员_modules中
      self._modules[str(idx)] = module
  
  def forward(self, X):
    # _modules类型为OrderedDict，保证了添加的顺序
    for block in self._modules.values():
      X = block(X)
    return X

In [13]:
net = MySequential(nn.Linear(20,256),nn.ReLU(),nn.Linear(256,10))
net(X)

tensor([[-0.0512, -0.0184,  0.3491, -0.0808,  0.5053,  0.2147,  0.0901, -0.1932,
          0.0550,  0.0875],
        [ 0.0121, -0.0745,  0.2227,  0.0008,  0.2554,  0.1676,  0.1401, -0.2116,
          0.0744,  0.1664]], grad_fn=<AddmmBackward0>)

## 更多灵活性

在FixedHiddenMLP中，实现一个隐藏层，其权重(`self.rand_weight`)会在实例化时被随机初始化为常量，并不会被反向传播更新。

在返回输出前，运行了一个循环，在L1范数>1的条件下，将输出向量/2，直到满足条件，所以可以把任意代码集成到神经网络的计算中。  

各种块都可以任意组合。

In [ ]:
class FixedHiddenMLP(nn.Module):
  def __init__(self):
    super().__init__()
    # 不计算梯度的随机权重参数，在训练期间保持不变
    self.rand_weight = torch.rand((20,20),requires_grad=False)
    self.linear = nn.Linear(20,20)
  
  def forward(self,X):
    X = self.linear(X)
    #使用常量参数
    X = F.relu(torch.mm(X,self.rand_weight)+1)
    # 复用全连接层，共享参数
    X = self.linear(X)
    # 控制流
    while X.abs().sum() >1:
      X /= 2
    return X.sum() 